# Click Prediction

Predict if a given bid request is likely to be clicked or not, based on the prior distribution of features.

Train data is 1/4 of the training set.
Test data is the entire validation set.

In [1]:
import numpy as np
import re
import xgboost
import pandas as pd

df = pd.read_csv("dataset/train.csv")
df_valid = pd.read_csv("dataset/validation.csv")
df_test = pd.read_csv("dataset/test.csv")

In [2]:
train_split = np.array_split(df, 4)
train = train_split[0]
train_y = train['click']

cols = ['weekday', 'hour', 'bidid', 'logtype', 'userid', 'useragent',
       'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid',
       'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat',
       'slotprice', 'creative', 'bidprice','payprice', 'keypage',
       'advertiser', 'usertag']

train_x = train[cols]

In [3]:
features = ['weekday', 'hour', 'city', 'slotprice', 'bidprice','payprice', 'advertiser']
X = train[features]
y = train_y

model = xgboost.XGBClassifier()
model.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [4]:
test_split = np.array_split(df_valid, 1)
test = test_split[0]
test_x = test[features]
test_x
test_predictions = model.predict(test_x)
test_x['click_pred'] = test_predictions
test_x['click'] = test['click']
test_x['abs_diff'] = abs(test_x['click_pred'] - test_x['click'])
test_x[['click_pred', 'click', 'abs_diff']]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,click_pred,click,abs_diff
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,0,0,0
9,0,0,0


In [5]:
test_x['abs_diff'].mean()

0.00075396415000550463

In [7]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_x['click'], test_x['click_pred'])
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.92%
